In [1]:
%%capture
%cd ..

In [2]:
# Import data

import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [47]:
ds_root = 'dataset/soh_charge'
colors = ['red', 'orange', 'blue', 'green', 'purple', 'lightblue']


In [48]:

plt.figure(figsize=(8, 6))
plt_charge = plt.subplot(1, 1, 1)

file_names = []


for file_name in os.listdir(ds_root):
    if file_name.startswith('soh'):
        curr_path = f'{ds_root}/{file_name}'

        df_output = pd.read_csv(curr_path)
        cs2x_part = curr_path.split('-')[1].split('.csv')[0]

        valori_percentuali = (df_output['SOH'] / 1.1) * 100
        color = colors[len(plt_charge.lines) % len(colors)]

        plt_charge.plot(valori_percentuali, label=f'{cs2x_part}', linestyle='-', color=color)
        file_names.append(f'soh-{cs2x_part}.csv')

# Pulisci il plot prima di tracciare i nuovi dati
plt_charge.clear()

# Controllo e rimozione delle righe con valori di SOH che si discostano troppo
for file_name in file_names:
    
    df_output = pd.read_csv(f'{ds_root}/{file_name}')

    valori_percentuali = (df_output['SOH'] / 1.1) * 100
    color = colors[len(plt_charge.lines) % len(colors)]

    plt_charge.plot(valori_percentuali, label=f'{file_name}', linestyle='-', color=color)

plt_charge.set_title('Charge SOH trend')
plt_charge.set_xlabel('Cycle')
plt_charge.set_ylabel('SOH(%)')

plt.xlim(left=0)
plt_charge.grid(True)
plt_charge.legend(loc='upper right', bbox_to_anchor=(1.2, 1.))

plt.tight_layout()
plt.show()

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Lista dei colori che vuoi utilizzare per le curve
colors = ['red', 'orange', 'blue', 'green', 'purple', 'lightblue']

# Inizializza il plot per SOH discharge
plt.figure()

# Lista per i nomi dei file SOH discharge
file_names_discharge = []

# Dizionario per memorizzare i risultati di ogni file
results_dict = {}

# Loop attraverso i file in ds_root
for i, file_name in enumerate(os.listdir(ds_root)):
    file_path = os.path.join(ds_root, file_name)
    
    if file_name.startswith('discharge'):
        df = pd.read_csv(file_path)

        first_step_index_7 = df[df['Step_Index'] == 7].groupby('Cycle_Index').head(1).index
        last_step_index_7 = df[df['Step_Index'] == 7].groupby('Cycle_Index').tail(1).index

        # Utilizza l'indice corretto (5 anziché 6)
        first1 = [df.iloc[i, 5] for i in first_step_index_7]
        last1 = [df.iloc[i, 5] for i in last_step_index_7]

        if not first1 or not last1 or len(first1) != len(last1):
            print(f"Errore: cicli non validi nel file {file_name}")
            continue

        result1 = [b - a for a, b in zip(first1, last1)]

        filtered_vector_discharge = [result1[0]]
        for i in range(1, len(result1)):
            difference = abs(result1[i] - result1[i - 1])
            if difference <= 0.01:
                filtered_vector_discharge.append(result1[i])

        cs2x_part = file_name.split('charge-')[1].split('.csv')[0]
        file_names_discharge.append(cs2x_part)

        # Salva i risultati in un file CSV nella cartella "soh_discharge"
        # output_csv_path = os.path.join(soh_discharge_folder, f'soh_{cs2x_part}.csv')

        # Creazione del DataFrame
        df_soh = pd.DataFrame({'Cycle': range(1, len(filtered_vector_discharge) + 1),
                               'SOH': filtered_vector_discharge})

        # Controllo e rimozione delle righe con deviazioni eccessive
        df_soh['SOH_Diff'] = df_soh['SOH'].diff()
        threshold = 0.1  # Puoi regolare il valore di soglia a seconda della tua necessità

        # Filtraggio delle righe con deviazioni eccessive
        df_soh_filtered = df_soh[abs(df_soh['SOH_Diff']) <= threshold]

        # Eliminazione della colonna temporanea 'SOH_Diff'
        df_soh_filtered = df_soh_filtered.drop(columns=['SOH_Diff'])

        # Salvataggio del DataFrame filtrato nel file CSV
        # df_soh_filtered.to_csv(output_csv_path, index=False)

        # print(f"File CSV discharge creato con successo: {output_csv_path}")

        # Salva i risultati nel dizionario solo se non sono già presenti
        if cs2x_part not in results_dict:
            results_dict[cs2x_part] = df_soh_filtered['SOH'].tolist()

        # Ottieni l'indice del colore utilizzando il modulo
        color_index = len(file_names_discharge) - 1
        color = colors[color_index % len(colors)]

        # Traccia la curva utilizzando un colore dalla lista colors
        plt.plot(df_soh_filtered['SOH'], label=f'{cs2x_part}', linestyle='-', color=color)

# Aggiungi titoli e legenda
plt.title('Discharge SOH trend')
plt.xlabel('Cycle')
plt.ylabel('SOH(%)')
plt.xlim(left=0)
plt.grid(True)

# Sposta la legenda in alto a destra senza che esca dalla figura
plt.legend(file_names_discharge, loc='upper right', bbox_to_anchor=(1.2, 1.))

# Mostra il plot
plt.show()